In [15]:
#抽出コード
import os
import random
import csv
import torch
import torchvision.transforms as T
from torchvision import models
from PIL import Image
from tqdm import tqdm
import shutil

# === 設定 ===
base_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\use_finish\imagenetv2-top-images\imagenetv2-top-images-format-val"
output_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50"#出力
output_csv = os.path.join(output_dir, "clean_dataset_labels.csv")
num_samples = 50 #画像数
confidence_threshold = 0.7 #確信度

os.makedirs(output_dir, exist_ok=True)

# === モデルとデバイス ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
model = model.to(device).eval()
imagenet_labels = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]

# === 前処理 ===
preprocess = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# === 全画像リスト作成 ===
all_images = []
for class_id in os.listdir(base_dir):
    class_dir = os.path.join(base_dir, class_id)
    if not os.path.isdir(class_dir): continue
    try:
        class_idx = int(class_id)
    except ValueError:
        continue
    for img_name in os.listdir(class_dir):
        if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
            all_images.append((os.path.join(class_dir, img_name), class_idx))

print(f"📸 総画像数: {len(all_images)}")

# === 推論と抽出 ===
selected = []
random.shuffle(all_images)

for img_path, true_idx in tqdm(all_images, desc="Selecting confident samples"):
    img = Image.open(img_path).convert("RGB")
    x = preprocess(img).unsqueeze(0).to(device)
    with torch.no_grad():
        outputs = model(x)
        probs = torch.nn.functional.softmax(outputs, dim=1)
        top1_prob, top1_idx = torch.max(probs, 1)

    if top1_idx.item() == true_idx and top1_prob.item() >= confidence_threshold:
        selected.append((img_path, true_idx, imagenet_labels[true_idx], top1_prob.item()))

    if len(selected) >= num_samples:
        break

print(f"✅ 選ばれた画像数: {len(selected)}")

# === 保存 ===
rows = []
for i, (src, label_id, label_name, prob) in enumerate(selected):
    new_name = f"sample{i+1:04d}.jpg"
    dst = os.path.join(output_dir, new_name)
    shutil.copy(src, dst)
    rows.append([new_name, label_id, label_name, f"{prob:.4f}"])

with open(output_csv, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["image", "label_id", "label_name", "confidence"])
    writer.writerows(rows)

print(f"\n🎯 高信頼度データセットを作成しました → {output_dir}")
print(f"🧾 ラベルCSV: {output_csv}")


📸 総画像数: 10000


Selecting confident samples:   1%|          | 80/10000 [00:01<02:30, 66.07it/s]

✅ 選ばれた画像数: 50

🎯 高信頼度データセットを作成しました → C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50
🧾 ラベルCSV: C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50\clean_dataset_labels.csv


In [18]:
#検証コード
import os
import torch
import pandas as pd
import torchvision.transforms as T
from torchvision import models
from PIL import Image
from tqdm import tqdm

# === 設定 ===
csv_path = r"C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50\clean_dataset_labels.csv"
img_dir = r"C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50"

# === モデル ===
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
model = model.to(device).eval()
imagenet_labels = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]

# === 前処理 ===
preprocess = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# === CSV読込 ===
df = pd.read_csv(csv_path)
df.columns = df.columns.str.strip().str.lower()

# === 推論して一致判定 ===
results = []
correct = 0
total = 0

for _, row in tqdm(df.iterrows(), total=len(df), desc="Validating"):
    filename = row["image"]
    true_id = int(row["label_id"])
    true_label = imagenet_labels[true_id]
    img_path = os.path.join(img_dir, filename)
    if not os.path.exists(img_path):
        continue

    try:
        img = Image.open(img_path).convert("RGB")
        x = preprocess(img).unsqueeze(0).to(device)

        with torch.no_grad():
            outputs = model(x)
            probs = torch.nn.functional.softmax(outputs, dim=1)
            top1_prob, top1_idx = torch.max(probs, 1)
            pred_label = imagenet_labels[top1_idx.item()]
            pred_id = top1_idx.item()
            prob = top1_prob.item()

        is_correct = (pred_id == true_id)
        results.append([filename, true_id, true_label, pred_id, pred_label, f"{prob:.3f}", is_correct])
        total += 1
        correct += int(is_correct)

    except Exception as e:
        print(f"⚠️ {filename} の検証中にエラー: {e}")

# === 結果まとめ ===
accuracy = correct / total * 100 if total > 0 else 0
print(f"\n✅ 検証完了: {total} 枚")
print(f"🎯 一致率 (Top-1): {accuracy:.2f}%")

# === 結果保存 ===
output_path = os.path.join(os.path.dirname(csv_path), "label_validation_results.csv")
pd.DataFrame(
    results,
    columns=["image", "true_id", "true_label", "pred_id", "pred_label", "probability", "is_match"]
).to_csv(output_path, index=False, encoding="utf-8")

print(f"📄 検証結果を保存しました → {output_path}")


Validating: 100%|██████████| 50/50 [00:00<00:00, 73.89it/s]


✅ 検証完了: 50 枚
🎯 一致率 (Top-1): 100.00%
📄 検証結果を保存しました → C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50\label_validation_results.csv


In [3]:
#攻撃前のtensorを作るコード

import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from pathlib import Path

# === 元画像と出力先 ===
input_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3")
output_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\data3_clean_tensor")
os.makedirs(output_dir, exist_ok=True)

# === transform：AE生成前と整合 (ToTensorのみ) ===
to_tensor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # → [0,1]
])

for fname in tqdm(sorted(os.listdir(input_dir))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img = Image.open(input_dir / fname).convert("RGB")
    tensor = to_tensor(img).unsqueeze(0)  # [1,3,H,W]
    torch.save(tensor, output_dir / (Path(fname).stem + ".pt"))


100%|██████████| 1002/1002 [00:06<00:00, 155.43it/s]


In [19]:
#攻撃前のtensorを作るコード

import os
import torch
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
from pathlib import Path

# === 元画像と出力先 ===
input_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\clean_dataset_50")
output_dir = Path(r"C:\Users\sit\wavelet_CGAN\testdata3\dataset_50_tensor")
os.makedirs(output_dir, exist_ok=True)

# === transform：AE生成前と整合 (ToTensorのみ) ===
to_tensor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),  # → [0,1]
])

for fname in tqdm(sorted(os.listdir(input_dir))):
    if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img = Image.open(input_dir / fname).convert("RGB")
    tensor = to_tensor(img).unsqueeze(0)  # [1,3,H,W]
    torch.save(tensor, output_dir / (Path(fname).stem + ".pt"))


100%|██████████| 52/52 [00:00<00:00, 213.06it/s]


In [3]:
import os
import random
import csv
import torch
import torchvision.transforms as T
from torchvision import models
from PIL import Image
from tqdm import tqdm
import shutil
import pandas as pd
from pathlib import Path
import warnings

# === 共通設定 ===
# 警告の抑制 (MobileNetV2のweightsの仕様変更に関する警告など)
warnings.filterwarnings("ignore", category=UserWarning)

# --- パス設定 (必要に応じて変更してください) ---
BASE_DIR = Path(r"C:\Users\sit\Downloads\imagenetv2-matched-frequency\imagenetv2-matched-frequency-format-val")
OUTPUT_ROOT_DIR = Path(r"C:\Users\sit\wavelet_CGAN\colab\data2") # 抽出データセットの出力先 (検証・Tensor生成の入力元)

# --- 抽出/データセット作成 設定 ---
NUM_SAMPLES = 300 # 抽出する画像数
CONFIDENCE_THRESHOLD = 0.8 # 確信度の閾値
OUTPUT_CSV = OUTPUT_ROOT_DIR / "clean_dataset_labels.csv"

# --- Tensorファイル出力設定 ---
TENSOR_OUTPUT_DIR_2 = OUTPUT_ROOT_DIR.parent / f"{OUTPUT_ROOT_DIR.name}_tensor"# Tensor出力先

# === 共通コンポーネントの初期化 ===
# デバイス設定
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# モデル (MobileNetV2)
MODEL = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1)
MODEL = MODEL.to(DEVICE).eval()
IMAGENET_LABELS = models.MobileNet_V2_Weights.IMAGENET1K_V1.meta["categories"]

# モデル用前処理 (抽出・検証で使用)
PREPROCESS = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Tensor作成用前処理 (ToTensorのみ)
TO_TENSOR_ONLY = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),  # → [0,1]
])

# === 1. 高信頼度画像の抽出と保存 ===
def extract_and_save_confident_images():
    """
    指定されたディレクトリから、モデルが高確信度で正しく分類した画像を抽出し、
    新しいディレクトリにコピーしてCSVファイルを作成する。
    """
    print("--- 1. 高信頼度画像の抽出と保存 ---")
    os.makedirs(OUTPUT_ROOT_DIR, exist_ok=True)
    
    # 全画像リスト作成
    all_images = []
    for class_id in os.listdir(BASE_DIR):
        class_dir = BASE_DIR / class_id
        if not class_dir.is_dir(): continue
        try:
            class_idx = int(class_id)
        except ValueError:
            continue
        for img_name in os.listdir(class_dir):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                all_images.append((class_dir / img_name, class_idx))

    print(f"📸 総画像数: {len(all_images)}")

    # 推論と抽出
    selected = []
    random.shuffle(all_images)

    for img_path, true_idx in tqdm(all_images, desc="Selecting confident samples"):
        try:
            img = Image.open(img_path).convert("RGB")
            x = PREPROCESS(img).unsqueeze(0).to(DEVICE)
            with torch.no_grad():
                outputs = MODEL(x)
                probs = torch.nn.functional.softmax(outputs, dim=1)
                top1_prob, top1_idx = torch.max(probs, 1)

            if top1_idx.item() == true_idx and top1_prob.item() >= CONFIDENCE_THRESHOLD:
                selected.append((img_path, true_idx, IMAGENET_LABELS[true_idx], top1_prob.item()))

            if len(selected) >= NUM_SAMPLES:
                break
        except Exception as e:
            # ファイル破損等でエラーが発生した場合はスキップ
            # print(f"⚠️ {img_path} 処理中にエラー: {e}") 
            continue

    print(f"✅ 選ばれた画像数: {len(selected)}")

    # 保存
    rows = []
    for i, (src, label_id, label_name, prob) in enumerate(selected):
        new_name = f"sample{i+1:04d}.jpg"
        dst = OUTPUT_ROOT_DIR / new_name
        shutil.copy(src, dst)
        rows.append([new_name, label_id, label_name, f"{prob:.4f}"])

    with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["image", "label_id", "label_name", "confidence"])
        writer.writerows(rows)

    print(f"🎯 高信頼度データセットを作成しました → {OUTPUT_ROOT_DIR}")
    print(f"🧾 ラベルCSV: {OUTPUT_CSV}")
    print("-" * 50)
    return len(selected) > 0 # 成功/失敗フラグ

# === 2. ラベルの再検証 ===
def validate_labels():
    """
    抽出されたデータセットのCSVに基づき、再度モデルで推論を行い、
    記録されたラベルとの一致率（Top-1）を検証する。
    """
    if not OUTPUT_CSV.exists():
        print(f"⚠️ 検証用CSVファイルが見つかりません: {OUTPUT_CSV}")
        return

    print("--- 2. ラベルの再検証 ---")
    
    # CSV読込
    df = pd.read_csv(OUTPUT_CSV)
    df.columns = df.columns.str.strip().str.lower()
    
    # 推論して一致判定
    results = []
    correct = 0
    total = 0

    for _, row in tqdm(df.iterrows(), total=len(df), desc="Validating"):
        filename = row["image"]
        true_id = int(row["label_id"])
        true_label = IMAGENET_LABELS[true_id]
        img_path = OUTPUT_ROOT_DIR / filename
        if not img_path.exists():
            continue

        try:
            img = Image.open(img_path).convert("RGB")
            x = PREPROCESS(img).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                outputs = MODEL(x)
                probs = torch.nn.functional.softmax(outputs, dim=1)
                top1_prob, top1_idx = torch.max(probs, 1)
                pred_label = IMAGENET_LABELS[top1_idx.item()]
                pred_id = top1_idx.item()
                prob = top1_prob.item()

            is_correct = (pred_id == true_id)
            results.append([filename, true_id, true_label, pred_id, pred_label, f"{prob:.3f}", is_correct])
            total += 1
            correct += int(is_correct)

        except Exception as e:
            print(f"⚠️ {filename} の検証中にエラー: {e}")

    # 結果まとめ
    accuracy = correct / total * 100 if total > 0 else 0
    print(f"\n✅ 検証完了: {total} 枚")
    print(f"🎯 一致率 (Top-1): {accuracy:.2f}%")

    # 結果保存
    output_path = OUTPUT_ROOT_DIR / "label_validation_results.csv"
    pd.DataFrame(
        results,
        columns=["image", "true_id", "true_label", "pred_id", "pred_label", "probability", "is_match"]
    ).to_csv(output_path, index=False, encoding="utf-8")

    print(f"📄 検証結果を保存しました → {output_path}")
    print("-" * 50)

# === 3. 攻撃前のTensorファイル作成 ===
def create_tensors(input_dir: Path, output_dir: Path, tensor_transform):
    """
    指定された画像ディレクトリ内の画像をPyTorch Tensorに変換し、指定されたディレクトリに保存する。
    """
    print(f"--- 3. Tensorファイル作成: {output_dir.name} ---")
    os.makedirs(output_dir, exist_ok=True)
    
    if not input_dir.exists():
        print(f"⚠️ 入力ディレクトリが見つかりません: {input_dir}")
        return

    for fname in tqdm(sorted(os.listdir(input_dir)), desc=f"Creating tensors for {output_dir.name}"):
        if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        try:
            img = Image.open(input_dir / fname).convert("RGB")
            tensor = tensor_transform(img).unsqueeze(0)  # [1,3,H,W]
            torch.save(tensor, output_dir / (Path(fname).stem + ".pt"))
        except Exception as e:
            print(f"⚠️ {fname} のTensor作成中にエラー: {e}")

    print(f"💾 Tensorファイルを保存しました → {output_dir}")
    print("-" * 50)

# === メイン処理 ===
if __name__ == "__main__":
    
    # 1. 画像の抽出と保存
    is_extracted = extract_and_save_confident_images()
    
    if is_extracted:
        # 2. ラベルの再検証
        validate_labels()

        # 3. Tensorファイル作成（★ dataset_50_tensor のみ）
        create_tensors(OUTPUT_ROOT_DIR, TENSOR_OUTPUT_DIR_2, TO_TENSOR_ONLY)

    else:
        print("処理を続行できません。高信頼度の画像が選択されませんでした。")



--- 1. 高信頼度画像の抽出と保存 ---
📸 総画像数: 10000


Selecting confident samples:   9%|▊         | 864/10000 [00:13<02:26, 62.53it/s]


✅ 選ばれた画像数: 300
🎯 高信頼度データセットを作成しました → C:\Users\sit\wavelet_CGAN\colab\data2
🧾 ラベルCSV: C:\Users\sit\wavelet_CGAN\colab\data2\clean_dataset_labels.csv
--------------------------------------------------
--- 2. ラベルの再検証 ---


Validating: 100%|██████████| 300/300 [00:04<00:00, 74.14it/s]



✅ 検証完了: 300 枚
🎯 一致率 (Top-1): 100.00%
📄 検証結果を保存しました → C:\Users\sit\wavelet_CGAN\colab\data2\label_validation_results.csv
--------------------------------------------------
--- 3. Tensorファイル作成: data2_tensor ---


Creating tensors for data2_tensor: 100%|██████████| 302/302 [00:01<00:00, 190.07it/s]

💾 Tensorファイルを保存しました → C:\Users\sit\wavelet_CGAN\colab\data2_tensor
--------------------------------------------------
